In [84]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff

# Song fields
- name
- music_name
- stepfile_name
- offset
- beat_length
- bpm
- notes (Array["0101", "M201"...])

In [92]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        raw_notes = pd.read_csv('data/{0}_notes_generated.csv'.format(key), converters={'0': lambda x: str(x)}).values
        notes = [row[0] for row in raw_notes]
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.notes = notes
        self.extension = music_file.split('.')[1]

In [93]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_notes_generated.csv'.format(key) in save_files:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])
songs

{'In The Groove~Anubis': <__main__.SongFile at 0x118837c88>}

In [94]:
def write_song_header(output_stepfile, song):
    keys = ['VERSION', 'TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH']
    
    header_info = {
        'VERSION': 0.82,
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("//---------------dance-single - ----------------", file=output_stepfile)
    keys = ['NOTEDATA', 'CHARTNAME', 'STEPSTYPE', 'DIFFICULTY', 'METER', 'RADARVALUES', 'BPMS']
    print (song.bpm)
    step_info = {
        'NOTEDATA': '',
        'CHARTNAME': song.name,
        'STEPSTYPE': 'dance-single',
        'DIFFICULTY': 'Expert',
        'METER': 1,
        'RADARVALUES': '0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0,0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0',
        'BPMS': '0={:.3f}'.format(song.bpm)
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(step_info[key])), file=output_stepfile)
    
def write_notes(output_stepfile, song):
    print ("#NOTES:", file=output_stepfile)
    
    for i in range(len(song.notes)):
        row = song.notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == 0 and i != 0:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    write_notes(output_stepfile, song)
    output_stepfile.close()

In [95]:
song.stepfile_name = 'StepMania/Songs/In The Groove/Anubis/Anubis.ssc'

In [96]:
song = songs['In The Groove~Anubis']
step_song(song)

110.0
